In [67]:
import arcpy
from arcpy import env
import os
import glob
from arcgis import GIS
from arcgis.features import GeoAccessor
from arcgis.features import GeoSeriesAccessor
import pandas as pd

arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "90%"

# show all columns
pd.options.display.max_columns = None

# pd.DataFrame.spatial.from_featureclass(???)  
# df.spatial.to_featureclass(location=???,sanitize_columns=False)  

# gsa = arcgis.features.GeoSeriesAccessor(df['SHAPE'])  
# df['AREA'] = gsa.area  # KNOW YOUR UNITS

In [68]:
# fill NA values in Spatially enabled dataframes (ignores SHAPE column)
def fill_na_sedf(df_with_shape_column, fill_value=0):
    if 'SHAPE' in list(df_with_shape_column.columns):
        df = df_with_shape_column.copy()
        shape_column = df['SHAPE'].copy()
        del df['SHAPE']
        return df.fillna(fill_value).merge(shape_column,left_index=True, right_index=True, how='inner')
    else:
        raise Exception("Dataframe does not include 'SHAPE' column")

In [69]:
if not os.path.exists('Outputs'):
    os.makedirs('Outputs')
    
outputs = ['.\\Outputs', "REMM_Forecast_Review.gdb"]
gdb = os.path.join(outputs[0], outputs[1])
if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs[0], outputs[1])

In [70]:
taz_shp = pd.DataFrame.spatial.from_featureclass(r".\Ancillary\TAZ_900.gdb\taz_utm12")
taz_se = glob.glob(os.path.join(r'.\SE_Data','SE_*.csv'))

In [71]:
# csv = taz_se[0]
# year = os.path.splitext(os.path.basename(csv))[0].split('_')[-1]

# # read in csv and format
# df = pd.read_csv(taz_se[0])
# df.rename({';TAZID':'TAZID'}, inplace=True, axis=1)
# df = df[['TAZID', 'TOTHH', 'HHPOP', 'HHSIZE', 'TOTEMP', 'RETEMP', 'INDEMP', 'OTHEMP', 'ALLEMP', 'RETL', 'FOOD', 'MANU', 'WSLE', 'OFFI', 'GVED', 'HLTH', 'OTHR']].copy()

# # rename columns
# data_cols = ['TOTHH', 'HHPOP', 'HHSIZE', 'TOTEMP', 'RETEMP', 'INDEMP', 'OTHEMP', 'ALLEMP', 'RETL', 'FOOD', 'MANU', 'WSLE', 'OFFI', 'GVED', 'HLTH', 'OTHR']
# new_data_cols = [f'{col}_{year[2:4]}' for col in data_cols]
# myDict = { k:v for (k,v) in zip(data_cols, new_data_cols)}
# df.rename(myDict, inplace=True, axis=1)

# # new columns/ratios ???
# df[f'HHJOBI_{year[2:4]}'] = (df[f'TOTHH_{year[2:4]}'] * 1.8 ) + df[f'TOTEMP_{year[2:4]}']

# df

In [72]:
base = taz_shp[taz_shp['REMM']==1].copy()
base_cols = ['TAZID', 'ACRES', 'DEVACRES', 'CO_NAME', 'DISTMED', 'CityArea', 'OtherArea', 'SHAPE']
data_cols = ['TOTHH', 'HHPOP', 'HHSIZE', 'TOTEMP', 'RETEMP', 'INDEMP', 'OTHEMP', 'ALLEMP', 'RETL', 'FOOD', 'MANU', 'WSLE', 'OFFI', 'GVED', 'HLTH', 'OTHR', 'HHJOBI']
base = base[base_cols].copy()


for csv in taz_se:

    year = os.path.splitext(os.path.basename(csv))[0].split('_')[-1]
    # read in csv and format
    df = pd.read_csv(csv)
    df.rename({';TAZID':'TAZID'}, inplace=True, axis=1)
    df = df[['TAZID', 'TOTHH', 'HHPOP', 'HHSIZE', 'TOTEMP', 'RETEMP', 'INDEMP', 'OTHEMP', 'ALLEMP', 'RETL', 'FOOD', 'MANU', 'WSLE', 'OFFI', 'GVED', 'HLTH', 'OTHR']].copy()

    # new columns/ratios ???
    df['HHJOBI'] = (df['TOTHH'] * 1.8 ) + df['TOTEMP']

    # rename columns
    new_data_cols = [f'{col}_{year[2:4]}' for col in data_cols]
    myDict = { k:v for (k,v) in zip(data_cols, new_data_cols)}
    df.rename(myDict, inplace=True, axis=1)

    

    base = base.merge(df,on='TAZID', how='left')

# store all column names
all_columns = list(base.columns)

In [73]:
# The Decades
decade_cols = [col for col in all_columns if col.endswith(('_20','_30','_40','_50'))] # using tuple
# decade_years = [col for col in all_columns if any(yr in col for yr in ['_20','_30','_40','_50'])] # using "any" keyword
decade_cols.sort()
decade_export = base[base_cols + decade_cols].copy()
decade_export.spatial.to_featureclass(location=os.path.join(gdb, 'se_2020_2030_2040_2050'),sanitize_columns=False)

'e:\\Projects\\REMM-TAZ-SE-Review-For-MB\\Outputs\\scratch.gdb\\se_2020_2030_2040_2050'

In [74]:
# SE Categories
for col in data_cols:
    print(f'working on {col}...')
    se_cols = [c for c in all_columns if c.startswith(col)]
    se_cols.sort()
    se_export = base[base_cols + se_cols].copy()
    se_export.spatial.to_featureclass(location=os.path.join(gdb, f'SE_{col}_All_Years'),sanitize_columns=False)

working on TOTHH...
working on HHPOP...
working on HHSIZE...
working on TOTEMP...
working on RETEMP...
working on INDEMP...
working on OTHEMP...
working on ALLEMP...
working on RETL...
working on FOOD...
working on MANU...
working on WSLE...
working on OFFI...
working on GVED...
working on HLTH...
working on OTHR...


In [76]:
decade_export.spatial.to_featureclass(location=os.path.join(gdb, 'SE_2020_2030_2040_2050'),sanitize_columns=False)

'e:\\Projects\\REMM-TAZ-SE-Review-For-MB\\Outputs\\scratch.gdb\\SE_2020_2030_2040_2050'

# 2050 Policy Output 

In [175]:
zb = pd.read_csv(r".\Inputs\zoning_baseline_20221006.csv")
zpp = pd.read_csv(r".\Inputs\zoning_parcels_p_20221006.csv")
p = pd.read_csv(r".\Inputs\parcels_20221006.csv")

In [176]:
# format zpp
zpp_most_recent = zpp.sort_values('year', ascending=False).drop_duplicates('parcel_id').sort_index()
zpp_most_recent['year'] = 2019
# zpp_most_recent.to_csv(os.path.join(outputs[0], 'zpp_most_recent.csv'), index=False)
zpp_most_recent_ids = zpp_most_recent['parcel_id'].to_list()
del zpp_most_recent['TAZID_900']
del zpp_most_recent['locnote']
del zpp_most_recent['AreaName']
del zpp_most_recent['mponote']

In [177]:
# format zb, get most recent year
lu = {'t':1, 'f':0}

zb['type1'] = zb['type1'].map(lu)
zb['type2'] = zb['type2'].map(lu)
zb['type3'] = zb['type3'].map(lu)
zb['type4'] = zb['type4'].map(lu)
zb['type5'] = zb['type5'].map(lu)
zb['type6'] = zb['type6'].map(lu)
zb['type7'] = zb['type7'].map(lu)
zb['type8'] = zb['type8'].map(lu)

zb = zb[zb['parcel_id'].isin(zpp_most_recent_ids) == False].copy()

del zb['CO_NAME']
del zb['max_height']

In [178]:
# combine zoning and join with the parcels
zb_2050 = pd.concat([zb,zpp_most_recent])
# zb_2050.to_csv(os.path.join(outputs[0], 'zb_2050.csv'), index=False)
p = p[['parcel_id', 'TAZID_900', 'parcel_acres']].copy()
pzb_2050 = p.merge(zb_2050, on='parcel_id', how='inner')

pzb_2050['max_dua'].fillna(0, inplace=True)
pzb_2050['max_far'].fillna(0, inplace=True)

pzb_2050.shape

(712236, 14)

# Capacity Summary by TAZ

In [179]:
pzb_2050['du'] = pzb_2050['parcel_acres'] * pzb_2050['max_dua']
pzb_2050['fa'] = pzb_2050['parcel_acres'] * pzb_2050['max_far']

pzb_2050['du'] = pzb_2050['du'].round(2)
pzb_2050['fa'] = pzb_2050['fa'].round(2)

capacity_by_taz  = pzb_2050.groupby('TAZID_900')[['du', 'fa']].sum().reset_index()

mean_capacity_by_taz = pzb_2050.groupby('TAZID_900')[['max_dua', 'max_far']].mean().reset_index()
mean_capacity_by_taz.columns = ['TAZID_900','mean_max_dua', 'mean_max_far']
median_capacity_by_taz = pzb_2050.groupby('TAZID_900')[['max_dua', 'max_far']].median().reset_index()
median_capacity_by_taz.columns = ['TAZID_900','med_max_dua', 'med_max_far']

base = taz_shp[taz_shp['REMM']==1][['SA_TAZID', 'ACRES','DEVACRES','SHAPE']].copy()
base.rename({'SA_TAZID':'TAZID_900'}, axis=1, inplace=True)

capacity_by_taz = (base.merge(capacity_by_taz, on='TAZID_900', how='left')
                       .merge(mean_capacity_by_taz, on='TAZID_900', how='left')
                       .merge(median_capacity_by_taz, on='TAZID_900', how='left'))

capacity_by_taz['dua_taz'] = capacity_by_taz['du'] / capacity_by_taz['DEVACRES']
capacity_by_taz['far_taz'] = capacity_by_taz['fa'] / capacity_by_taz['DEVACRES']

pzb_2050['dua_taz'] = pzb_2050['du'].round(2)
pzb_2050['far_taz'] = pzb_2050['fa'].round(2)

capacity_by_taz.spatial.to_featureclass(location=os.path.join(gdb, 'Capacity_Summary_By_TAZ'),sanitize_columns=False)

'e:\\Projects\\REMM-TAZ-SE-Review-For-MB\\Outputs\\scratch.gdb\\Capacity_Summary_By_TAZ'

# zoning types summary by taz

In [180]:
# single family
acres_type1_by_taz = pzb_2050[pzb_2050['type1']==1].groupby('TAZID_900', )[['parcel_acres']].sum().reset_index()
acres_type1_by_taz.columns = ['TAZID_900','sf_acres']

# multifamily
acres_type2_by_taz = pzb_2050[pzb_2050['type2']==1].groupby('TAZID_900')[['parcel_acres']].sum().reset_index()
acres_type2_by_taz.columns = ['TAZID_900','mf_acres']

# residential
acres_res_by_taz = pzb_2050[(pzb_2050['type1']==1) | (pzb_2050['type2']==1)].groupby('TAZID_900')[['parcel_acres']].sum().reset_index()
acres_res_by_taz.columns = ['TAZID_900','res_acres']

# industrial
acres_type3_by_taz = pzb_2050[pzb_2050['type3']==1].groupby('TAZID_900')[['parcel_acres']].sum().reset_index()
acres_type3_by_taz.columns = ['TAZID_900','ind_acres']

# retail
acres_type4_by_taz = pzb_2050[pzb_2050['type4']==1].groupby('TAZID_900')[['parcel_acres']].sum().reset_index()
acres_type4_by_taz.columns = ['TAZID_900','ret_acres']

# office
acres_type5_by_taz = pzb_2050[pzb_2050['type5']==1].groupby('TAZID_900')[['parcel_acres']].sum().reset_index()
acres_type5_by_taz.columns = ['TAZID_900','off_acres']

# commercial
acres_com_by_taz = pzb_2050[(pzb_2050['type3']==1) | (pzb_2050['type4']==1) | (pzb_2050['type5']==1)].groupby('TAZID_900')[['parcel_acres']].sum().reset_index()
acres_com_by_taz.columns = ['TAZID_900','com_acres']
acres_com_by_taz['com_acres'].fillna(0, inplace=True)

base = taz_shp[taz_shp['REMM']==1][['SA_TAZID', 'ACRES','DEVACRES','SHAPE']].copy()
base.rename({'SA_TAZID':'TAZID_900'}, axis=1, inplace=True)
acres_type_by_taz = (base.merge(acres_type1_by_taz, on='TAZID_900', how='left')
                         .merge(acres_type2_by_taz, on='TAZID_900', how='left')
                         .merge(acres_type3_by_taz, on='TAZID_900', how='left')
                         .merge(acres_type4_by_taz, on='TAZID_900', how='left')
                         .merge(acres_type5_by_taz, on='TAZID_900', how='left')
                         .merge(acres_res_by_taz, on='TAZID_900', how='left')
                         .merge(acres_com_by_taz, on='TAZID_900', how='left'))

# fill NAs
acres_type_by_taz = fill_na_sedf(acres_type_by_taz, 0)

# calulate proportions get acreage from TAZ
acres_type_by_taz['res_acres_proportion'] = acres_type_by_taz['res_acres'] / acres_type_by_taz['DEVACRES']
acres_type_by_taz['com_acres_proportion'] = acres_type_by_taz['com_acres'] / acres_type_by_taz['DEVACRES']

acres_type_by_taz.spatial.to_featureclass(location=os.path.join(gdb, 'Buildable_Types_Summary_By_TAZ'),sanitize_columns=False)

'e:\\Projects\\REMM-TAZ-SE-Review-For-MB\\Outputs\\scratch.gdb\\Buildable_Types_Summary_By_TAZ'

# Housing Units by decade

In [181]:
b = pd.read_csv(r".\Inputs\buildings_20221006.csv")
b_2030 = pd.read_csv(r".\Inputs\run773year2030allbuildings.csv")
b_2040 = pd.read_csv(r".\Inputs\run773year2040allbuildings.csv")
b_2050 = pd.read_csv(r".\Inputs\run773year2050allbuildings.csv")

# calculate the decade
def add_built_decade(df):

    df['BUILT_DECADE'] = 'NA'
    df.loc[(df['year_built'] >= 1840) & (df['year_built'] < 1850), 'BUILT_DECADE'] = "1840's"
    df.loc[(df['year_built'] >= 1850) & (df['year_built'] < 1860), 'BUILT_DECADE'] = "1850's"
    df.loc[(df['year_built'] >= 1860) & (df['year_built'] < 1870), 'BUILT_DECADE'] = "1860's"
    df.loc[(df['year_built'] >= 1870) & (df['year_built'] < 1880), 'BUILT_DECADE'] = "1870's"
    df.loc[(df['year_built'] >= 1880) & (df['year_built'] < 1890), 'BUILT_DECADE'] = "1880's"
    df.loc[(df['year_built'] >= 1890) & (df['year_built'] < 1900), 'BUILT_DECADE'] = "1890's"
    df.loc[(df['year_built'] >= 1900) & (df['year_built'] < 1910), 'BUILT_DECADE'] = "1900's"
    df.loc[(df['year_built'] >= 1910) & (df['year_built'] < 1920), 'BUILT_DECADE'] = "1910's"
    df.loc[(df['year_built'] >= 1920) & (df['year_built'] < 1930), 'BUILT_DECADE'] = "1920's"
    df.loc[(df['year_built'] >= 1930) & (df['year_built'] < 1940), 'BUILT_DECADE'] = "1930's"
    df.loc[(df['year_built'] >= 1940) & (df['year_built'] < 1950), 'BUILT_DECADE'] = "1940's"
    df.loc[(df['year_built'] >= 1950) & (df['year_built'] < 1960), 'BUILT_DECADE'] = "1950's"
    df.loc[(df['year_built'] >= 1960) & (df['year_built'] < 1970), 'BUILT_DECADE'] = "1960's"
    df.loc[(df['year_built'] >= 1970) & (df['year_built'] < 1980), 'BUILT_DECADE'] = "1970's"
    df.loc[(df['year_built'] >= 1980) & (df['year_built'] < 1990), 'BUILT_DECADE'] = "1980's"
    df.loc[(df['year_built'] >= 1990) & (df['year_built'] < 2000), 'BUILT_DECADE'] = "1990's"
    df.loc[(df['year_built'] >= 2000) & (df['year_built'] < 2010), 'BUILT_DECADE'] = "2000's"
    df.loc[(df['year_built'] >= 2010) & (df['year_built'] < 2020), 'BUILT_DECADE'] = "2010's"
    df.loc[(df['year_built'] >= 2020) & (df['year_built'] < 2030), 'BUILT_DECADE'] = "2020's"
    df.loc[(df['year_built'] >= 2030) & (df['year_built'] < 2040), 'BUILT_DECADE'] = "2030's"
    df.loc[(df['year_built'] >= 2040) & (df['year_built'] < 2050), 'BUILT_DECADE'] = "2040's"
    df.loc[(df['year_built'] >= 2050) & (df['year_built'] < 2060), 'BUILT_DECADE'] = "2050's"

    return df

b = add_built_decade(b) 
b_2030 = add_built_decade(b_2030) 
b_2040 = add_built_decade(b_2040) 
b_2050 = add_built_decade(b_2050)  

In [182]:
p_taz = p[['parcel_id', 'TAZID_900']].copy()
base = taz_shp[taz_shp['REMM']==1][['SA_TAZID', 'ACRES','DEVACRES','SHAPE']].copy()
base.rename({'SA_TAZID':'TAZID_900'}, axis=1, inplace=True)

In [183]:
b_res = b[b['building_type_id'].isin([1,2])==True].copy()
b_res = b_res.merge(p_taz, on='parcel_id', how='left')

b_2030_res = b_2030[b_2030['building_type_id'].isin([1,2])==True].copy()
b_2030_res = b_2030_res.merge(p_taz, on='parcel_id', how='left')

b_2040_res = b_2040[b_2040['building_type_id'].isin([1,2])==True].copy()
b_2040_res = b_2040_res.merge(p_taz, on='parcel_id', how='left')

b_2050_res = b_2050[b_2050['building_type_id'].isin([1,2])==True].copy()
b_2050_res = b_2050_res.merge(p_taz, on='parcel_id', how='left')

# 1990's
sf_units_built_90s_by_taz = b_res[(b_res['building_type_id']==1) & (b_res['BUILT_DECADE']=="1990's")].groupby('TAZID_900')[['residential_units']].sum().reset_index()
sf_units_built_90s_by_taz.columns = ['TAZID_900','sf_units_built_1990s']

mf_units_built_90s_by_taz = b_res[(b_res['building_type_id']==2) & (b_res['BUILT_DECADE']=="1990's")].groupby('TAZID_900')[['residential_units']].sum().reset_index()
mf_units_built_90s_by_taz.columns = ['TAZID_900','mf_units_built_1990s']

res_units_built_90s_by_taz = b_res[(b_res['BUILT_DECADE']=="1990's")].groupby('TAZID_900')[['residential_units']].sum().reset_index()
res_units_built_90s_by_taz.columns = ['TAZID_900','res_units_built_1990s']

# 2000's
sf_units_built_00s_by_taz = b_res[(b_res['building_type_id']==1) & (b_res['BUILT_DECADE']=="2000's")].groupby('TAZID_900')[['residential_units']].sum().reset_index()
sf_units_built_00s_by_taz.columns = ['TAZID_900','sf_units_built_2000s']

mf_units_built_00s_by_taz = b_res[(b_res['building_type_id']==2) & (b_res['BUILT_DECADE']=="2000's")].groupby('TAZID_900')[['residential_units']].sum().reset_index()
mf_units_built_00s_by_taz.columns = ['TAZID_900','mf_units_built_2000s']

res_units_built_00s_by_taz = b_res[(b_res['BUILT_DECADE']=="2000's")].groupby('TAZID_900')[['residential_units']].sum().reset_index()
res_units_built_00s_by_taz.columns = ['TAZID_900','res_units_built_2000s']

# 2010's
sf_units_built_10s_by_taz = b_res[(b_res['building_type_id']==1) & (b_res['BUILT_DECADE']=="2010's")].groupby('TAZID_900')[['residential_units']].sum().reset_index()
sf_units_built_10s_by_taz.columns = ['TAZID_900','sf_units_built_2010s']

mf_units_built_10s_by_taz = b_res[(b_res['building_type_id']==2) & (b_res['BUILT_DECADE']=="2010's")].groupby('TAZID_900')[['residential_units']].sum().reset_index()
mf_units_built_10s_by_taz.columns = ['TAZID_900','mf_units_built_2010s']

res_units_built_10s_by_taz = b_res[(b_res['BUILT_DECADE']=="2010's")].groupby('TAZID_900')[['residential_units']].sum().reset_index()
res_units_built_10s_by_taz.columns = ['TAZID_900','res_units_built_2010s']

# 2020's
sf_units_built_20s_by_taz = b_res[(b_res['building_type_id']==1) & (b_res['BUILT_DECADE']=="2020's")].groupby('TAZID_900')[['residential_units']].sum().reset_index()
sf_units_built_20s_by_taz.columns = ['TAZID_900','sf_units_built_2020s']

mf_units_built_20s_by_taz = b_res[(b_res['building_type_id']==2) & (b_res['BUILT_DECADE']=="2020's")].groupby('TAZID_900')[['residential_units']].sum().reset_index()
mf_units_built_20s_by_taz.columns = ['TAZID_900','mf_units_built_2020s']

res_units_built_20s_by_taz = b_res[(b_res['BUILT_DECADE']=="2020's")].groupby('TAZID_900')[['residential_units']].sum().reset_index()
res_units_built_20s_by_taz.columns = ['TAZID_900','res_units_built_2020s']

# REMM 2020's
remm_sf_units_built_20s_by_taz = b_2030_res[(b_2030_res['building_type_id']==1) & (b_2030_res['BUILT_DECADE']=="2020's")].groupby('TAZID_900')[['residential_units']].sum().reset_index()
remm_sf_units_built_20s_by_taz.columns = ['TAZID_900','REMM_sf_units_built_2020s']

remm_mf_units_built_20s_by_taz = b_2030_res[(b_2030_res['building_type_id']==2) & (b_2030_res['BUILT_DECADE']=="2020's")].groupby('TAZID_900')[['residential_units']].sum().reset_index()
remm_mf_units_built_20s_by_taz.columns = ['TAZID_900','REMM_mf_units_built_2020s']

remm_res_units_built_20s_by_taz = b_2030_res[(b_2030_res['BUILT_DECADE']=="2020's")].groupby('TAZID_900')[['residential_units']].sum().reset_index()
remm_res_units_built_20s_by_taz.columns = ['TAZID_900','REMM_res_units_built_2020s']

# REMM 2030's
remm_sf_units_built_30s_by_taz = b_2040_res[(b_2040_res['building_type_id']==1) & (b_2040_res['BUILT_DECADE']=="2030's")].groupby('TAZID_900')[['residential_units']].sum().reset_index()
remm_sf_units_built_30s_by_taz.columns = ['TAZID_900','REMM_sf_units_built_2030s']

remm_mf_units_built_30s_by_taz = b_2040_res[(b_2040_res['building_type_id']==2) & (b_2040_res['BUILT_DECADE']=="2030's")].groupby('TAZID_900')[['residential_units']].sum().reset_index()
remm_mf_units_built_30s_by_taz.columns = ['TAZID_900','REMM_mf_units_built_2030s']

remm_res_units_built_30s_by_taz = b_2040_res[(b_2040_res['BUILT_DECADE']=="2030's")].groupby('TAZID_900')[['residential_units']].sum().reset_index()
remm_res_units_built_30s_by_taz.columns = ['TAZID_900','REMM_res_units_built_2030s']

# REMM 2040's
remm_sf_units_built_40s_by_taz = b_2050_res[(b_2050_res['building_type_id']==1) & (b_2050_res['BUILT_DECADE']=="2040's")].groupby('TAZID_900')[['residential_units']].sum().reset_index()
remm_sf_units_built_40s_by_taz.columns = ['TAZID_900','REMM_sf_units_built_2040s']

remm_mf_units_built_40s_by_taz = b_2050_res[(b_2050_res['building_type_id']==2) & (b_2050_res['BUILT_DECADE']=="2040's")].groupby('TAZID_900')[['residential_units']].sum().reset_index()
remm_mf_units_built_40s_by_taz.columns = ['TAZID_900','REMM_mf_units_built_2040s']

remm_res_units_built_40s_by_taz = b_2050_res[(b_2050_res['BUILT_DECADE']=="2020's")].groupby('TAZID_900')[['residential_units']].sum().reset_index()
remm_res_units_built_40s_by_taz.columns = ['TAZID_900','REMM_res_units_built_2040s']


res_units_built_by_taz = (base.merge(sf_units_built_90s_by_taz, on='TAZID_900', how='left')
                              .merge(mf_units_built_90s_by_taz, on='TAZID_900', how='left')
                              .merge(res_units_built_90s_by_taz, on='TAZID_900', how='left')
                              .merge(sf_units_built_00s_by_taz, on='TAZID_900', how='left')
                              .merge(mf_units_built_00s_by_taz, on='TAZID_900', how='left')
                              .merge(res_units_built_00s_by_taz, on='TAZID_900', how='left')
                              .merge(sf_units_built_10s_by_taz, on='TAZID_900', how='left')
                              .merge(mf_units_built_10s_by_taz, on='TAZID_900', how='left')
                              .merge(res_units_built_10s_by_taz, on='TAZID_900', how='left')
                              .merge(sf_units_built_20s_by_taz, on='TAZID_900', how='left')
                              .merge(mf_units_built_20s_by_taz, on='TAZID_900', how='left')
                              .merge(res_units_built_20s_by_taz, on='TAZID_900', how='left')
                              .merge(remm_sf_units_built_20s_by_taz, on='TAZID_900', how='left')
                              .merge(remm_mf_units_built_20s_by_taz, on='TAZID_900', how='left')
                              .merge(remm_res_units_built_20s_by_taz, on='TAZID_900', how='left')
                              .merge(remm_sf_units_built_30s_by_taz, on='TAZID_900', how='left')
                              .merge(remm_mf_units_built_30s_by_taz, on='TAZID_900', how='left')
                              .merge(remm_res_units_built_30s_by_taz, on='TAZID_900', how='left')
                              .merge(remm_sf_units_built_40s_by_taz, on='TAZID_900', how='left')
                              .merge(remm_mf_units_built_40s_by_taz, on='TAZID_900', how='left')
                              .merge(remm_res_units_built_40s_by_taz, on='TAZID_900', how='left'))

# fill NAs
res_units_built_by_taz = fill_na_sedf(res_units_built_by_taz, 0)

res_units_built_by_taz.spatial.to_featureclass(location=os.path.join(gdb, 'Residential_Units_Built_By_TAZ'),sanitize_columns=False)

'e:\\Projects\\REMM-TAZ-SE-Review-For-MB\\Outputs\\scratch.gdb\\Residential_Units_Built_By_TAZ'